In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [2]:
import logging
from pathlib import Path

import holoviews as hv
import hvplot.polars  # noqa
import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
import polars as pl
import scipy
import scipy.signal as signal
from icecream import ic
from numba import njit
from polars import col

from src.data.database_manager import DatabaseManager
from src.data.quality_checks import check_sample_rate
from src.features.pupil import _get_blink_segments, add_blink_threshold, filter_pupil
from src.features.transforming import map_trials
from src.log_config import configure_logging
from src.plots.utils import prepare_multiline_hvplot

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [3]:
db = DatabaseManager()

In [4]:
with db:
    df = db.get_table("Raw_Pupil")
df = add_blink_threshold(df)

In [8]:
blink_segments = _get_blink_segments(df)
blink_segments

# blink_segments.hvplot.hist(
#     y="duration",
#     bins=10000,
#     xlabel="Duration (ms)",
#     ylabel="Frequency",
#     subplots=True,
#     by="participant_id",
#     width=300,
#     height=200,
#     xlim=(0, 500),
#     xaxis="bare",
#     yaxis="bare",
# )

pupil,start_timestamp,end_timestamp,trial_id,participant_id,trial_number,duration
str,f64,f64,i64,i64,i64,f64
"""r""",298103.0866,298136.477,1,1,1,33.3904
"""l""",298103.0866,298153.0962,1,1,1,50.0096
"""r""",365974.541,366057.9282,1,1,1,83.3872
"""r""",379255.133,379288.5074,1,1,1,33.3744
"""l""",402412.8274,402529.6914,1,1,1,116.864
"""r""",402429.5858,402546.317,1,1,1,116.7312
…,…,…,…,…,…,…
"""l""",2.6642e6,2.6644e6,332,28,12,183.5008
"""r""",2.6642e6,2.6644e6,332,28,12,183.5216


In [6]:
df.hvplot(
    x="timestamp", y="pupil_r_thresholded", groupby="trial_id", width=800, height=400
)
# NOTE: blinks are detected properly for all participants

BokehModel(combine_events=True, render_bundle={'docs_json': {'d79a72a0-8298-4f48-a160-7b8ab82b8862': {'version…

In [7]:
median filt3r

SyntaxError: invalid syntax (248996880.py, line 1)

In [7]:
filter_pupil(
    df,
    "pupil_r_thresholded",
    sample_rate=60,
    highcut=0.2,
    order=6,
).hvplot(
    x="timestamp",
    y="pupil_r_thresholded_filtered",
    groupby="trial_id",
    width=800,
    height=400,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'6ca3cf60-7542-42d0-9f71-27ad76feeb47': {'version…